# Completions Endpoint

### Completions endpoint is used for single turn tasks. Such tasks does not require multiple turns to get the final answer, for example, classification tasks.

In [ ]:
from dotenv import load_dotenv
import os
from openai import OpenAI

# Load environment variables from .env file
load_dotenv()

# Access the API key from environment variables
api_key = os.getenv('OPENAI_API_KEY')
client = OpenAI(api_key=api_key)


### Understanding Tokens
In English:
1 token ~ 4 characters
100 tokens ~ 75 words

Example: 150 words -> nax_tokens = 200

### Understading Temperature Parameter

Output of LLMs  is often non-deterministic. You can control such behavior using `temperature`.
It ranges from 0 to 2. where 0 mean fully deterministic output and 2 is fully random output. 
The default value of it is 1.


In [ ]:
def get_response(prompt, temperature = 1 , max_tokens = 30 ,model = "gpt-3.5-turbo-instruct"):
    return client.completions.create(
    model = model,
    prompt = prompt,
    temperature = temperature,
    max_tokens=max_tokens
)


### To convert the response object into dictionary use : `response.model_dump()`

In [ ]:
prompt = "What is the OpenAI API?"




### Text Completions for classification tasks usin In-context Learning

In [ ]:
prompt = """
Classify the sentiment of the following sentences give the three labels: negative, positive, or neutral.
1- Unbelievably good!
2- Shoes fell apart on the second use.
3- The shoes look nice, but they aren't very comfortable.
4- Can't wait to show them off!
"""
response = get_response(prompt)
print(response.choices[0].text.strip())

# Chat Completions Endpoint

### There are 3 main roles:
- systems: to control the response behavior
- user: represents the user message/query to be sent
- assistant: represents the model response for the user query

In [ ]:
def get_response_chat(messages, model = "gpt-3.5-turbo"):
    return client.chat.completions.create(
    model = model,
    messages = messages
)

In [ ]:
chat_messages = [{"role": "system", "content": "you are a python tutor"}]
# examples for the few-shot learning
user_qa = ["What does the type() function do?", "how to get the minimum number in a mutable object?"]

for q in user_qa:
    print(f"User: {q} \n")
    user_dict = {"role": "user" , "content": q}
    chat_messages.append(user_dict)
    response = get_response_chat(messages=chat_messages)
    assistant_dict = {"role": "assistant", "content": response.choices[0].message.content}
    print(f"Assistant: {response.choices[0].message.content} \n")
    
    
    

# Text Moderation

### The process of identifying inappropriate content and it is a time consuming one. we can automate this using OpenAPI.
the response contains 3 main dicts:
- categories: `true/false` indicator of category violation
- category_score: 0 ~ 1 float value confidence of a violation, do not refer as probability
- flagged `true/false` indicator of a violation

In [ ]:
def get_response_moderation(input, model = "text-moderation-latest"):
    return client.moderations.create(
        model=model,
        input=input
    )
    


In [ ]:
input = "I will kill this burger sandwitch."
response = get_response_moderation(input)
print(response.results[0].category_scores)

# Audio models - Whisper

In [ ]:
# audio_file = open("sample.mb3", "rb")
def get_transcript (file, model = "whisper-1"):
    return client.audio.transcriptions.create(
        model= model,
        file = file
    )

In [ ]:
audio_file = open("sample.mp3", "rb")
response = get_transcript(audio_file)
print(response.text)

## Translation endpoints - only works from language X to english.

In [37]:
# prompt used to give extra context to advance the translation/transcripts
def get_translation (file, model = "whisper-1", prompt = None):
    return client.audio.translations.create(
            model= model,
            file = file,
            prompt = prompt
        )

# Combining Models together - Model Chaining

Model chaining is the process of feeding the output of a model X as an input to a model Y where X,Y can be same or different models.